# Imports



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
from sklearn.impute import SimpleImputer

# Compiling Data



In [2]:
df_emissions = pd.read_csv('Emissions.csv')
df_fossil_fuels = pd.concat([pd.read_csv('Fossil Fuels A.csv'), pd.read_csv('Fossil Fuels B.csv')])

In [3]:
# Remove unused columns
df_emissions = df_emissions.rename(columns={'OBS_VALUE': 'Greenhouse Gas Emissions'}).drop(['DATAFLOW', 'FREQ', 'INDICATOR', 'UNIT'], axis=1)
df_fossil_fuels.drop(['DATAFLOW', 'FREQ', 'UNIT_MULT', 'UNIT_MEASURE', 'OBS_STATUS'], axis=1, inplace=True)

# Create dictionary to map country codes and abbreviations
df_codes = pd.read_csv('Country_Codes.csv')
codes = df_codes['M49 Code']
abbreviations = df_codes['ISO-alpha3 Code']
code_dict = dict(zip(codes, abbreviations))

# Remap energy stats areas from numbers to abbreviations
df_fossil_fuels = df_fossil_fuels.replace({'REF_AREA': code_dict})

In [4]:
# Add fossil fuel data
fossil_fuel_codes = {100: 'Hard Coal', 3000: 'Natural Gas', 4100: 'Crude Oil', 4652: 'Motor Gasoline', 4670: 'Gas Oil/Diesel Oil'}

# production
tmp = df_fossil_fuels.loc[df_fossil_fuels['TRANSACTION'] == 1]
for code in fossil_fuel_codes:
    tmp2 = tmp.loc[tmp['COMMODITY'] == code]
    tmp2 = tmp2.drop(['COMMODITY', 'TRANSACTION'], axis=1).rename(columns={'OBS_VALUE': f'{fossil_fuel_codes[code]} Production'})
    df_emissions = pd.merge(df_emissions, tmp2, on=['REF_AREA', 'TIME_PERIOD'], how='left')

# import
tmp = df_fossil_fuels.loc[df_fossil_fuels['TRANSACTION'] == 3]
for code in fossil_fuel_codes:
    tmp2 = tmp.loc[tmp['COMMODITY'] == code]
    tmp2 = tmp2.drop(['COMMODITY', 'TRANSACTION'], axis=1).rename(columns={'OBS_VALUE': f'{fossil_fuel_codes[code]} Import'})
    df_emissions = pd.merge(df_emissions, tmp2, on=['REF_AREA', 'TIME_PERIOD'], how='left')

# export
tmp = df_fossil_fuels.loc[df_fossil_fuels['TRANSACTION'] == 4]
for code in fossil_fuel_codes:
    tmp2 = tmp.loc[tmp['COMMODITY'] == code]
    tmp2 = tmp2.drop(['COMMODITY', 'TRANSACTION'], axis=1).rename(columns={'OBS_VALUE': f'{fossil_fuel_codes[code]} Export'})
    df_emissions = pd.merge(df_emissions, tmp2, on=['REF_AREA', 'TIME_PERIOD'], how='left')

df = df_emissions.rename(columns={'TIME_PERIOD': 'Year'})

# Reference area is unecessary for our analysis
df.drop(columns=['REF_AREA'], axis=1, inplace=True)

tmp = df.pop('Greenhouse Gas Emissions')
df.insert(0, 'Greenhouse Gas Emissions', tmp)

# EDA



In [5]:
#df = pd.read_csv('./complete_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1290 entries, 0 to 1289
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Greenhouse Gas Emissions       1290 non-null   float64
 1   Year                           1290 non-null   int64  
 2   Hard Coal Production           451 non-null    float64
 3   Natural Gas Production         747 non-null    float64
 4   Crude Oil Production           724 non-null    float64
 5   Motor Gasoline Production      879 non-null    float64
 6   Gas Oil/Diesel Oil Production  879 non-null    float64
 7   Hard Coal Import               1015 non-null   float64
 8   Natural Gas Import             921 non-null    float64
 9   Crude Oil Import               883 non-null    float64
 10  Motor Gasoline Import          1050 non-null   float64
 11  Gas Oil/Diesel Oil Import      1059 non-null   float64
 12  Hard Coal Export               661 non-null    f

In [6]:
# Missing value analysis
columns = list(df.columns)
missing_vals = []
for col in columns:
    missing_vals.append(df[col].isna().sum())

fig = px.bar(x=columns, y=missing_vals, title='Missing Value Analysis')
fig.update_layout(xaxis_title='Column', yaxis_title='Missing Values')
fig.show()

In [7]:
missing_counts = np.arange(stop=18)
row_counts = []
for i in missing_counts:
    row_counts.append(len(df[df.isnull().sum(axis=1) == i].index))

fig = px.bar(x=missing_counts, y=row_counts, title='Row Counts Missing Different Numbers of Values')
fig.update_layout(xaxis_title='Number of Missing Values', yaxis_title='Number of Rows')

fig.write_html('Charts/missing_val_row_counts.html')
fig.show()

In [8]:
# Energy consumption data is highly colinear with emissions, and not that interesting to explore
# df = df.drop(['Net Electricity Production', 'Final Energy Consumption', 'Energy Consumption Manufacturing', 'Energy Consumption Transportation', 'Energy Consumption Other', 'Total Energy Supply'], axis=1)

In [9]:
# plot distributions
# ax = df['Energy Consumption Manufacturing'].plot.hist(bins=12, alpha=0.5)
# NOT A GAUSSIAN/NORMAL DISTRIBUTION

# Preprocessing and Cleaning



In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1290 entries, 0 to 1289
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Greenhouse Gas Emissions       1290 non-null   float64
 1   Year                           1290 non-null   int64  
 2   Hard Coal Production           451 non-null    float64
 3   Natural Gas Production         747 non-null    float64
 4   Crude Oil Production           724 non-null    float64
 5   Motor Gasoline Production      879 non-null    float64
 6   Gas Oil/Diesel Oil Production  879 non-null    float64
 7   Hard Coal Import               1015 non-null   float64
 8   Natural Gas Import             921 non-null    float64
 9   Crude Oil Import               883 non-null    float64
 10  Motor Gasoline Import          1050 non-null   float64
 11  Gas Oil/Diesel Oil Import      1059 non-null   float64
 12  Hard Coal Export               661 non-null    f

In [11]:
# remove rows with too many missing values
MAX_MISSING = 5
df.drop(df[df.isnull().sum(axis=1)>MAX_MISSING].index,axis=0,inplace=True)
# https://stackoverflow.com/questions/55207940/remove-the-missing-values-from-the-rows-having-greater-than-5-missing-values-and

# Missing value counts after dropping rows with many missing values 
columns = list(df.columns)
missing_vals_2 = []
for col in columns:
    missing_vals_2.append(df[col].isna().sum())

fig = px.bar(x=columns, y=missing_vals_2, title='Missing Value Analysis After Dropping Rows')
fig.update_layout(xaxis_title='Column', yaxis_title='Missing Values')
fig.show()

In [12]:
# Create combined figure for website
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("Initial Missing Values by Column", "Missing Values by Column After Dropping Rows"))
fig.add_trace(go.Bar(x=columns,
                         y=missing_vals),
              row=1, col=1)

fig.add_trace(go.Bar(x=columns,
                         y=missing_vals_2),
              row=1, col=2)

fig.update_yaxes(title_text="Missing Values", row=1, col=1)
fig.update_xaxes(title_text="Column", row=1, col=1)
fig.update_yaxes(title_text="Missing Values", row=1, col=2)
fig.update_xaxes(title_text="Column", row=1, col=2)



fig.update_layout(showlegend=False)
fig.write_html('Charts/missing_values.html')
fig.show()

In [13]:
# Show correlation matrix
fig = px.imshow(df.corr(),title="Correlation Matrix",width=800,height=800)
fig.update_xaxes(tickangle=-45)
# fig.write_html("Charts/full_cor_mat.html")
# fig.show()

In [14]:
df.drop(columns=['Hard Coal Export', 'Hard Coal Production', 'Natural Gas Production', 'Crude Oil Production'], axis=1, inplace=True)

# Show correlation matrix
fig = px.imshow(df.corr(),title="Correlation Matrix",width=800,height=800)
fig.update_xaxes(tickangle=-45)
fig.write_html("Charts/cor_mat.html")
fig.show()

In [15]:
fig = px.scatter_matrix(
    df,
    dimensions=['Motor Gasoline Production', 'Gas Oil/Diesel Oil Production', 'Crude Oil Import', 'Motor Gasoline Import'],
    color='Greenhouse Gas Emissions',
    width=900,
    height=900)

fig.update_layout(
    font=dict(
        size=8,
    )
)
# fig.update_traces(diagonal_visible=False)

fig.write_html('Charts/correlated_variables.html')
fig.show()

In [16]:
# Create combined figure for website
fig = make_subplots(rows=2, cols=2, shared_yaxes=True)

fig.add_trace(go.Scatter(x=df['Motor Gasoline Production'],
                         y=df['Greenhouse Gas Emissions'],
                 mode='markers',
                 name='Motor Gasoline Production'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df['Gas Oil/Diesel Oil Production'],
                         y=df['Greenhouse Gas Emissions'],
                 mode='markers',
                 name='Gas Oil/Diesel Oil Production'),
              row=1, col=2)
fig.add_trace(go.Scatter(x=df['Crude Oil Import'],
                         y=df['Greenhouse Gas Emissions'],
                 mode='markers',
                 name='Crude Oil Import'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=df['Motor Gasoline Import'],
                         y=df['Greenhouse Gas Emissions'],
                 mode='markers',
                 name='Motor Gasoline Import'),
              row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Motor Gasoline Production", row=1, col=1)
fig.update_xaxes(title_text="Gas Oil/Diesel Oil Production", row=1, col=2)
fig.update_xaxes(title_text="Crude Oil Import", row=2, col=1)
fig.update_xaxes(title_text="Motor Gasoline Import", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Emissions", row=1, col=1)
fig.update_yaxes(title_text="Emissions", row=1, col=2)
fig.update_yaxes(title_text="Emissions", row=2, col=1)
fig.update_yaxes(title_text="Emissions", row=2, col=2)

fig.update_layout(showlegend=False)

fig.write_html('Charts/emissions_vs_correlated_fuels.html')
fig.show()

In [17]:
df.drop(columns=['Gas Oil/Diesel Oil Production', 'Motor Gasoline Production', 'Motor Gasoline Import'], axis=1, inplace=True)

# Show correlation matrix
fig = px.imshow(df.corr(),title="Correlation Matrix With Key Features",width=700,height=700)
fig.update_xaxes(tickangle=-45)
fig.write_html("Charts/cor_mat_after_drop.html")
fig.show()

In [18]:
# fill in missing values and normalize
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
columns = list(df.columns)
# columns.remove('Greenhouse Gas Emissions')

for col in columns:
    # Replace missing values
    imputer = imputer.fit(df[[col]])
    df[col] = imputer.transform(df[[col]])
    # Normalize (no need to subtract min since minimum is 0)
    df[col] = (df[col] - df[col].abs().min())/(df[col].abs().max() - df[col].abs().min())

df.to_csv('complete_data.csv', index=False)